In [ ]:


import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Using TensorFlow backend.


/kaggle/input/fake-and-real-news-dataset/Fake.csv
/kaggle/input/fake-and-real-news-dataset/True.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
true=pd.read_csv('../input/fake-and-real-news-dataset/True.csv')
fake=pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv')


In [4]:
true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [5]:
fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [6]:
true['result']=1
fake['result']=0

In [7]:
true.head()

,title,text,subject,date,result
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [8]:
fake.head()

,title,text,subject,date,result
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [9]:
df=pd.concat([true,fake])

In [10]:
df.isna().sum()

title      0
text       0
subject    0
date       0
result     0
dtype: int64

In [11]:
df['text']=df['title']+""+df['text']+""+df['subject']
del df['title']
del df['date']
del df['subject']


In [12]:
df

,text,result
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1
...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0
23479,How to Blow $700 Million: Al Jazeera America F...,0


In [13]:
sentence = df['text'].values.tolist()
result= df['result'].values.tolist()

In [14]:

X_train, X_test, Y_train,Y_test= train_test_split(sentence, result, test_size=0.2)

In [15]:
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

In [16]:
tokenizer=Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(X_train)
padded_train=pad_sequences(sequences,5000,truncating='post')

In [17]:
sequences_test=tokenizer.texts_to_sequences(X_test)
padded_test=pad_sequences(sequences_test,5000,truncating='post')

In [18]:
padded_test.shape

(8980, 5000)

In [19]:
Y_test.shape

(8980,)

In [20]:
model= tf.keras.Sequential([
    tf.keras.layers.Embedding(10000,16,input_length=5000),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5000, 16)          160000    
_________________________________________________________________
flatten (Flatten)            (None, 80000)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 480006    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 640,013
Trainable params: 640,013
Non-trainable params: 0
_________________________________________________________________


In [22]:
history=model.fit(padded_train, Y_train, epochs=10, validation_data=(padded_test, Y_test))

Train on 35918 samples, validate on 8980 samples
Epoch 1/10
35918/35918 [==============================] - 27s 746us/sample - loss: 0.0729 - accuracy: 0.9732 - val_loss: 0.0019 - val_accuracy: 0.9998
Epoch 2/10
35918/35918 [==============================] - 24s 674us/sample - loss: 8.5399e-04 - accuracy: 0.9999 - val_loss: 6.5356e-04 - val_accuracy: 0.9999
Epoch 3/10
35918/35918 [==============================] - 25s 690us/sample - loss: 1.9084e-04 - accuracy: 1.0000 - val_loss: 4.1836e-04 - val_accuracy: 0.9998
Epoch 4/10
35918/35918 [==============================] - 24s 661us/sample - loss: 7.5011e-05 - accuracy: 1.0000 - val_loss: 3.2069e-04 - val_accuracy: 0.9999
Epoch 5/10
35918/35918 [==============================] - 23s 646us/sample - loss: 2.8669e-05 - accuracy: 1.0000 - val_loss: 2.5717e-04 - val_accuracy: 0.9999
Epoch 6/10
35918/35918 [==============================] - 24s 678us/sample - loss: 1.4836e-05 - accuracy: 1.0000 - val_loss: 2.1666e-04 - val_accuracy: 0.9999
Epoch

In [24]:
score = model.evaluate(padded_test, Y_test)

8980/8980 [==============================] - 2s 180us/sample - loss: 1.5886e-04 - accuracy: 0.9999


In [25]:
print('Test accuracy:', score[1])

Test accuracy: 0.99988866
